In [1]:
import numpy as np
import pandas as pd

# Task 0
Read the dataset from csv file & perform data cleaning - remove all rows, which contains `?` in some columns.
Also check for data correctness (salary & salary $K).

In [68]:
adult = pd.read_csv("../data/adult.csv", index_col=[0])
adult = adult[~adult.eq("?").any(1)]
adult.info()
adult.describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30162 entries, 0 to 32560
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             30162 non-null  int64 
 1   workclass       30162 non-null  object
 2   education       30162 non-null  object
 3   marital-status  30162 non-null  object
 4   occupation      30162 non-null  object
 5   relationship    30162 non-null  object
 6   race            30162 non-null  object
 7   sex             30162 non-null  object
 8   hours-per-week  30162 non-null  int64 
 9   native-country  30162 non-null  object
 10  salary          30162 non-null  object
 11  salary K$       30162 non-null  int64 
dtypes: int64(3), object(9)
memory usage: 3.0+ MB


,age,hours-per-week,salary K$
count,30162.000000,30162.000000,30162.000000
mean,38.437902,40.931238,73.968570
std,13.134665,11.979984,85.365144
min,17.000000,1.000000,15.000000
25%,28.000000,40.000000,26.000000
50%,37.000000,40.000000,38.000000
75%,47.000000,45.000000,49.000000
max,90.000000,99.000000,349.000000


In [74]:
df1 = adult[adult["salary K$"] > 50]
df2 = adult[adult["salary"].eq(">50K")]

are_equal = df1.equals(df2)
are_equal

True

# Task 1
Print the count of men and women in the dataset.

In [77]:
adult["sex"].value_counts()

Male      20380
Female     9782
Name: sex, dtype: int64

# Task 2
Find the average age of men in dataset

In [84]:
adult_men = adult[adult["sex"] == "Male"]
adult_men["age"].mean()

39.18400392541707

# Task 3
Get the percentage of people from Poland (native-country)

In [93]:
adult["native-country"].value_counts(normalize=True)["Poland"] * 100

0.18566408063125786

# Task 4
Get the mean and standard deviation of the age for people who earn > 50K per year. After this, get it for those who earn <= 50K.

In [99]:
adult_earn_more_50K = adult[adult["salary"] == ">50K"]
more_mean = adult_earn_more_50K["age"].mean()
more_std = adult_earn_more_50K["age"].std()
adult_earn_less_50K = adult[adult["salary"] == "<=50K"]
less_mean = adult_earn_less_50K["age"].mean()
less_std = adult_earn_less_50K["age"].std()
result = pd.DataFrame({
    "earn": ["more_50K", "less_50K"],
    "mean": [more_mean, less_mean],
    "std": [more_std, less_std]
})
result

,earn,mean,std
0,more_50K,43.95911,10.269633
1,less_50K,36.60806,13.464631


# Task 5
Check, if there are some people without higher education (education: Bachelors, Prof-school, Assoc-acdm, Assoc-voc, Masters, Doctorate), but with > 50K salary

In [133]:
adult[adult["salary"] == ">50K"]["education"].value_counts()

Bachelors       2126
HS-grad         1617
Some-college    1336
Masters          918
Prof-school      406
Assoc-voc        344
Doctorate        280
Assoc-acdm       256
10th              59
11th              59
7th-8th           35
12th              29
9th               25
5th-6th           12
1st-4th            6
Name: education, dtype: int64

# Task 6
Get the statistics of age for each type of education. Use `groupby` and `describe` for this.

In [113]:
adult.groupby("education")["age"].describe()

,count,mean,std,min,25%,50%,75%,max
education,,,,,,,,
10th,820.0,37.897561,16.225795,17.0,23.0,36.0,52.0,90.0
11th,1048.0,32.363550,15.089307,17.0,18.0,28.5,43.0,90.0
12th,377.0,32.013263,14.373710,17.0,19.0,28.0,41.0,79.0
1st-4th,151.0,44.622517,14.929051,19.0,33.0,44.0,56.0,81.0
5th-6th,288.0,41.649306,14.754622,17.0,28.0,41.0,53.0,82.0
7th-8th,557.0,47.631957,15.737479,17.0,34.0,49.0,60.0,90.0
9th,455.0,40.303297,15.335754,17.0,28.0,38.0,53.0,90.0
Assoc-acdm,1008.0,37.286706,10.509755,19.0,29.0,36.0,44.0,90.0
Assoc-voc,1307.0,38.246366,11.181253,19.0,30.0,37.0,45.0,84.0


# Task 7
Compare the married and non-married men salaries. Who earns more? (>50K or <=50K)
Married men are those, whom `marital-status` starts with "Married". Others are not.

In [157]:
salary_married = adult[(adult["sex"] == "Male") & (adult["marital-status"].str.contains("Married"))]["salary"].value_counts()
salary_non_married = adult[(adult["sex"] == "Male") & (~adult["marital-status"].str.contains("Married"))]["salary"].value_counts()
res = pd.DataFrame({
    "status": ["married", "non-married"],
    "salary <= 50K": [salary_married[0], salary_non_married[0]],
    "salary > 50K": [salary_married[1], salary_non_married[1]]
})
res

,status,salary <= 50K,salary > 50K
0,married,7052,5723
1,non-married,6932,673


# Task 8
Get the max hours per week some person works. How many people works the same amount of hours per week?

In [164]:
max_hours = adult["hours-per-week"].max()
max_hours

99

In [165]:
people_number = adult["hours-per-week"].value_counts()[max_hours]
people_number

78

# Task 9
Analyze the correlation between data in dataset. Understand connected fields in it and print highlight thier connection.

In [166]:
adult.corr()

,age,hours-per-week,salary K$
age,1.000000,0.101599,0.208203
hours-per-week,0.101599,1.000000,0.196378
salary K$,0.208203,0.196378,1.000000


There is a slight positive correlation (0.208) between salary and age (the older - the higher the salary), also between salary and the number of working hours per week (0.196) (the more hours - the higher the salary), and even more insignificant positive correlation (0.101) between age and the number of working hours (with increasing age working hours increase)